In [ ]:
# Notebook created by Mahdiyeh Behjat Khatooni
# Date: 2023

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn import datasets, metrics, svm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import metrics

from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from itertools import cycle
from sklearn.metrics import roc_auc_score

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

data preparation

In [ ]:
%cd drive/MyDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import gzip
# unzip data set
train_file = gzip.open('mnist/train-images-idx3-ubyte.gz','r')

image_size = 28
train_examples = 60000

import numpy as np
# train set
train_file.read(16)
train_buf = train_file.read()
train_data = np.frombuffer(train_buf, dtype=np.uint8).astype(np.float32)
train_data = train_data.reshape(train_examples, image_size, image_size)
train_vectors = []
for v in range(60000):
  train_vectors.append(train_data[v].reshape(-1))
  train_image = np.asarray(train_data[v]).squeeze()
  # if u want to show examples make free below lines
  #plt.imshow(train_image)
  #plt.show()
# I created feature vectors as a 1*784 vector for each example in data set
#plt.show(train_vectors[10])
print(len(train_vectors))

# test set
test_examples = 10000
test_file = gzip.open('mnist/t10k-images-idx3-ubyte.gz','r')
test_file.read(16)
test_buf = test_file.read()
test_data = np.frombuffer(test_buf, dtype=np.uint8).astype(np.float32)
test_data = test_data.reshape(test_examples , image_size, image_size)
test_vectors = []
for v in range(10000):
  test_vectors.append(test_data[v].reshape(-1))
  test_image = np.asarray(test_data[v]).squeeze()
  # if u want to show examples make free below lines
  #plt.imshow(test_image)
  #plt.show()
#print(test_vectors[10])

In [ ]:
# train labels
train_label = gzip.open('mnist/train-labels-idx1-ubyte.gz','r')
train_label.read(8)
label_buffer = train_label.read()
train_labels = np.frombuffer(label_buffer, dtype=np.uint8).astype(np.int64)

print(len(train_labels))
# test labels
test_label = gzip.open('mnist/t10k-labels-idx1-ubyte.gz','r')
test_label.read(8)
label_buffer = test_label.read()
test_labels = np.frombuffer(label_buffer, dtype=np.uint8).astype(np.int64)

print(test_labels)

In [ ]:
# Show some of the images
num_images_to_show = 10

plt.figure(figsize=(12, 6))
for i in range(num_images_to_show):
    plt.subplot(1, num_images_to_show, i+1)
    plt.imshow(train_data[i+10], cmap='gray')
    plt.title(f"Label: {train_labels[i+10]}")
    plt.axis('off')

plt.show()

In [ ]:
def select_samples_per_class(x_data, y_data, num_samples):
    selected_indices = []
    for class_label in np.unique(y_data):
        class_indices = np.where(y_data == class_label)[0]
        selected_indices.extend(np.random.choice(class_indices, num_samples, replace=False))
    x_data = np.array(x_data)
    return x_data[selected_indices], y_data[selected_indices]

# Choose 200 examples per class for the training set
num_train_samples_per_class = 500
x_train_selected, y_train_selected = select_samples_per_class(train_vectors, train_labels, num_train_samples_per_class)

# Choose 50 examples per class for the test subset
num_test_samples_per_class = 100
x_test_selected, y_test_selected = select_samples_per_class(test_vectors, test_labels, num_test_samples_per_class)
print(x_train_selected.shape)

In [8]:
# Shuffle the selected training data
shuffle_indices = np.random.permutation(len(x_train_selected))
x_train_subset = x_train_selected[shuffle_indices]
y_train_subset = y_train_selected[shuffle_indices]

# Split the test subset into features and labels
x_test_subset, y_test_subset = x_test_selected, y_test_selected


# # Normalize the pixel values to the range [0, 1]
x_train_subset = x_train_subset.astype('float32') / 255.0
x_test_subset = x_test_subset.astype('float32') / 255.0


In [11]:
# Split dataset into train and test sets
X_train, X_test, y_train, y_test = x_train_subset, x_test_subset, y_train_subset , y_test_subset

In [ ]:
# Split dataset into train and test sets
# X_train, X_test, y_train, y_test = train_vectors, test_vectors, train_labels , test_labels

In [ ]:
print(X_train.shape,X_test.shape)

In [21]:
# Perform PCA for dimensionality reduction
n_components = 100
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Crammer Singer Machine

Just set hyperparameters and start running

In [12]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
class CrammerSingerSVM:
  def __init__(self,X_train,Y_train,X_test,Y_test,threshold,max_iter,beta=1.0,epsilon=0.001,epsilon_0=0.001,kernel_function='RBF',gamma='scale',degree=5):
    self.beta = beta
    self.epsilon = epsilon
    self.epsilon_0 = epsilon_0
    self.kernel_function = kernel_function
    self.gamma = gamma
    self.degree = degree
    self.X_train = X_train
    self.X_test = X_test
    self.Y_test = Y_test
    self.threshold = threshold
    self.max_iter = max_iter


    classes = np.unique(Y_train)
    self.num_classes = len(classes)
    self.num_examples = self.X_train.shape[0]
    self.num_features = self.X_train.shape[1]

    self.Tau = np.zeros((self.num_examples,self.num_classes)) # i*r

    Y_train = np.array(Y_train).reshape(-1, 1)
    delta_train = OneHotEncoder(sparse_output=False).fit_transform(Y_train)     #one_hot_labels
    self.Y_train = delta_train

    self.F = (-1*self.beta) * self.Y_train

    self.A = []
    for index, row in enumerate(self.X_train):
      K_i_i = self.kernel(self.kernel_function,row , row)
      self.A.append(K_i_i)
    self.A = np.array(self.A) # a vector


  def kernel(self,kernel_function,a,b):

    match kernel_function:
      case 'linear':
        return np.dot(a,b.T)

      case 'polynomial':
        return (np.dot(a, b.T) + 1) ** self.degree

      case 'RBF':
        if self.gamma == 'scale':
          self.gamma = 1 / (self.num_features * self.X_train.var())
          print("gamma=",self.gamma)
        return np.exp(-self.gamma*np.linalg.norm(np.array(a) - np.array(b)) ** 2 )

  def runSVM(self):
    self.Tau = self.train_machine() #multipliers(weights)
    self.norms = np.linalg.norm(self.Tau, ord=2, axis=1)
    # non_zero_indices = np.argwhere((norms >= self.threshold).any())
    non_zero_indices = []
    for i, norm in enumerate(self.norms):
      print("pattern" , i , "norm:" , norm)
      if norm > self.threshold:
        non_zero_indices.append(i)

    self.support_weights = np.squeeze(self.Tau[non_zero_indices])
    self.support_patterns = np.squeeze(self.X_train[non_zero_indices])
    self.support_labels = np.squeeze(self.Y_train[non_zero_indices])
    # self.visualizePatterns()

    print("I am in Predict phase.")
    print("Number of Support Patterns" , len(non_zero_indices))

    H_matrix_train = []
    for i,ex in enumerate(self.X_train):
      H_matrix_train.append(self.predict(ex))
    H_matrix_train = np.array(H_matrix_train)

    H_matrix_test = []
    for i,ex in enumerate(self.X_test):
      H_matrix_test.append(self.predict(ex))

    H_matrix_test = np.array(H_matrix_test)
    return H_matrix_train , H_matrix_test # containing class for each test data point
    # return H_matrix_test


  def predict(self,x):
    # calculating confidence score for each label for sample x #
    num_support_patterns = self.support_patterns.shape[0]
    output_x = np.zeros(self.num_classes)
    for r in range(self.num_classes):
      summation = 0
      for i in range(num_support_patterns):
        summation = summation + (self.support_weights[i][r] * self.kernel(self.kernel_function,x,self.support_patterns[i]))
      output_x[r] = summation


    predicted_label = np.argmax(output_x)
    return predicted_label


  def train_machine(self):
    optimized = False
    index_holder = [] # a list
    iter = 0
    while not optimized:
      print("Round:" , iter)
      example_idx , psi = self.example_selection(self.X_train,self.Y_train,self.F,self.Tau)
      index_holder.append(example_idx)
      A_ex = self.A[example_idx] # a scalar
      label_ex = self.Y_train[example_idx]
      B_ex = self.calculate_B(example_idx,self.beta,self.X_train,label_ex,self.kernel_function,self.Tau)
      sum_D_r = 0
      D_ex = np.zeros(self.num_classes)
      for r in range(self.num_classes):
        D_r = np.array((self.F[example_idx][r] / A_ex) - self.Tau[example_idx][r] + label_ex[r])
        D_ex[r] = D_r

      sum_D_r = sum(D_ex)

      theta = ((1/self.num_classes) * sum_D_r) - (1/self.num_classes)
      Tau_ex_optimum = self.FixedPointAlgorithm(D_ex,theta,self.epsilon/2,A_ex,B_ex)
      difference_tau = Tau_ex_optimum - self.Tau[example_idx] # for example p
      self.F = self.update_F(self.F,difference_tau,self.kernel_function,example_idx,self.X_train)
      self.Tau[example_idx] = Tau_ex_optimum

      if psi[example_idx] < self.epsilon * self.beta or iter == self.max_iter:
      # or index_holder.count(example_idx) >= 3:
        optimized = True
      iter += 1

    return self.Tau #matrix containing weights of samples(support vectors)


  def example_selection(self,data,data_labels,F,Tau):

    psi_matrix = np.zeros(self.num_examples)
    for i , ex in enumerate(data):
      F_i = F[i] # a vector
      F_i_max = np.max(F_i) # maximum confidence score assigning to label r
      smaller_idx = []
      for r in range(self.num_classes):
        if Tau[i][r] < data_labels[i][r]:
          smaller_idx.append(r)
      F_i_min = np.min(F_i[smaller_idx]) # there must be at least one label r that satisfies this condition
      psi = F_i_max - F_i_min
      psi_matrix[i] = psi

    p_idx = np.argmax(psi_matrix)

    return p_idx , psi_matrix

  def FixedPointAlgorithm(self,D,theta,epsilon,A,B):
    l = 0
    theta_l = theta
    while True:
      l += 1
      max_array = np.zeros(self.num_classes)
      for r in range(self.num_classes):
        max_array[r] = max(theta_l,D[r])
      theta_l_prim = (1/self.num_classes * sum(max_array)) - 1 / self.num_classes
      if abs((theta_l - theta_l_prim) / theta_l) <= epsilon:
        break
      else:
        theta_l = theta_l_prim

    V = np.zeros(self.num_classes)
    for r in range(self.num_classes):
      b = min(theta_l_prim,D[r])
      V[r] = b

    Tau_star = V - (B / A)
    return Tau_star

  def calculate_B(self,p,beta,data,label_p,kernel_function,Tau):
    summation = np.zeros(self.num_classes)
    for i , x in enumerate(data):
      if i != p:
        holder = self.kernel(kernel_function,data[p],data[i])
        summation = summation + (holder * Tau[i]) # a row

    Bp = (-1 * beta * label_p) + summation  # a vector
    return Bp

  def update_F(self,F,difference_tau,kernel_function,p,data):
    for m in range(self.num_examples):
      kernel_result = self.kernel(kernel_function,data[p],data[m])

      for r in range(self.num_classes):
        F[m][r] = F[m][r] + (difference_tau[r] * kernel_result)

    return F

                                        ### Visualization ####
  def visualizePatterns(self):
    # min_max_scaler = preprocessing.MinMaxScaler()
    # X_normal = min_max_scaler.fit_transform(X)
    for x in self.X_train:
      if x not in self.support_patterns:
        plt.scatter(x[0], x[1] , c='blue',edgecolors='k',marker='o',)
      else:
        plt.scatter(x[0], x[1] , c = 'green',marker='x')
    plt.title("Multi Class SVM blob data with Support Patterns")
    return True

Build your instance

In [14]:
crammerSinger_instance = CrammerSingerSVM(X_train,y_train,X_test,y_test,threshold=0.0,max_iter = 2000,beta = 1e-2,epsilon =0.01,kernel_function='linear')

In [ ]:
y_pred_train , y_pred = crammerSinger_instance.runSVM()
# y_pred = crammerSinger_instance.runSVM()
print(y_pred)

In [ ]:
disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
disp.figure_.suptitle("Confusion Matrix")
plt.show()

# Train Accuracy
train_accuracy = accuracy_score(y_train, y_pred_train)
print(f"Train Accuracy: {train_accuracy:.2f}")
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
# precision
precision = precision_score(y_test , y_pred , average = 'micro')
print("precision :" , precision)
# recall
recall = recall_score(y_test, y_pred , average='micro')
print("recall : " , recall)
# F1 score
print("f1_score: " , f1_score(y_test, y_pred , average='micro'))

In [ ]:
# binarize the output
# remove this and at last calculate to not come binarize values before this part

binarize_prediction_values = label_binarize(y_pred, classes=[0, 1, 2 , 3 , 4 , 5 , 6 , 7 ,8 , 9])
binarizes_test_labels = label_binarize(y_test, classes=[0, 1, 2 , 3 , 4 , 5 , 6 , 7 ,8 , 9])
n_classes = 10

fpr = dict()
tpr = dict()
roc_auc = dict()
lw=2

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(binarizes_test_labels[:, i], binarize_prediction_values[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
colors = cycle(['blue', 'red', 'green','purple', 'orange', 'cyan','yellow', 'brown', 'magenta','black'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for multi-class data')
plt.legend(loc="lower right")
plt.show()


# AUC
AUC= roc_auc_score(binarizes_test_labels, binarize_prediction_values , multi_class="ovr", average="macro")
print("AUC:" , AUC)

# OVR single model

In [25]:
# Train a Logistic Regression classifier
clf = SVC(C=...,kernel='...',decision_function_shape='ovr')
clf.fit(X_train_pca, y_train)

# Predict on test data
y_pred = clf.predict(X_test_pca)

In [26]:
y_pred_train = clf.predict(X_train_pca)

In [ ]:
disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
disp.figure_.suptitle("Confusion Matrix")
plt.show()

# Accuracy
Acc_train = accuracy_score(y_train, y_pred_train)
print(f"Accuracy: {Acc_train:.2f}")
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
# precision
precision = precision_score(y_test , y_pred , average = 'micro')
print("precision :" , precision)
# recall
recall = recall_score(y_test, y_pred , average='micro')
print("recall : " , recall)
# F1 score
print("f1_score: " , f1_score(y_test, y_pred , average='micro'))

In [ ]:
# binarize the output
# remove this and at last calculate to not come binarize values before this part

binarize_prediction_values = label_binarize(y_pred, classes=[0, 1, 2 , 3 , 4 , 5 , 6 , 7 ,8 , 9])
binarizes_test_labels = label_binarize(y_test, classes=[0, 1, 2 , 3 , 4 , 5 , 6 , 7 ,8 , 9])
n_classes = 10

fpr = dict()
tpr = dict()
roc_auc = dict()
lw=2

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(binarizes_test_labels[:, i], binarize_prediction_values[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
colors = cycle(['blue', 'red', 'green','purple', 'orange', 'cyan','yellow', 'brown', 'magenta','black'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for multi-class data')
plt.legend(loc="lower right")
plt.show()


# AUC
AUC= roc_auc_score(binarizes_test_labels, binarize_prediction_values , multi_class="ovr", average="macro")
print("AUC:" , AUC)